# 台股財報擷取-綜合損益表

網址：[公開資訊觀測站/財務報表/採IFRS後/合併/個別報表/綜合損益表](https://mops.twse.com.tw/mops/web/t164sb04)

In [1]:
import requests, os, sqlite3
import pandas as pd

#輸入資產負債表擷取參數
print('擷取台股財報－綜合損益表...')
sid = input('請輸入個股代號：')
year = input('請輸入擷取年度（請輸入民國年，譬如：108）：')
season = input('請輸入擷取季（輸入格式及區間：01 ~ 04）：')

# 設定網址及 post 表單參數
url = 'https://mops.twse.com.tw/mops/web/t164sb04'
form_data = {
    'encodeURIComponent':1,
    'step':1,
    'firstin':1,
    'off':1,
    'queryName':'co_id',
    'inputType':'co_id',
    'TYPEK':'all',
    'isnew':'false',
    'co_id':sid,
    'year':year,
    'season':season,
}

# 擷取資料
try:
    data = requests.post(url, data=form_data)
except Exception as e:
    print('**WARRN: 擷取失敗')
    print('錯誤訊息', e)
else:
    # 使用 pandas 讀取網頁裡的表格資料
    dfs = pd.read_html(data.text)
    #dfs = pd.read_html('i2-income.html')

    # 取得綜合損益表表格資料
    df = dfs[10]

    #重設欄位名稱
    df.columns = list(range(df.shape[1]))

    # 設定索引值
    df = df.set_index(0)

    # 只取第一欄資料(即當季財報資料), 及刪除 nan 項
    df = df[1].dropna()

    # 只取1個欄位資料後, df 變成 series, 需轉換為 Dataframe
    df = pd.DataFrame(df)

    # 轉置表格, 即列與欄交換
    df = df.transpose()

    # 刪除重覆欄位
    df = df.loc[:, ~df.columns.duplicated()]

    # 新增股票代碼及財報日期欄位
    df['sid'] = sid
    df['year'] = year
    df['season'] = season

    # 設定索引值
    df = df.set_index(['sid', 'year', 'season'])

    # 設定資料庫連線
    conn = sqlite3.connect(os.path.join('data', 'stock.db'))

    # 判斷 income table 是否已存在
    query = "select count(*) from sqlite_master where type='table' and name='income'"
    exist = list(conn.execute(query))[0][0]

    # 如果 bs table 存在，新增資料，否則，建立表格及新增資料
    if exist:
        # 讀取資料庫中原有資料
        data = pd.read_sql('select * from income', conn, index_col=(['sid', 'year', 'season']))

        # 合併原有資料與新資料
        newdf = pd.concat([data, df], sort=True)

        # 刪除重覆項
        newdf.drop_duplicates(inplace=True)

        newdf.to_sql('income', conn, if_exists='replace')
        print(sid, year, '年第', season, '季綜合損益表新增成功')
    else:
        df.to_sql('income', conn, if_exists='replace')
        print(sid, year, '年第', season, '季綜合損益表儲存成功')

    # 關閉資料庫連線
    conn.close()